In [1]:
import os
import sys
import h5py
import json
import glob

import numpy as np
from sklearn.model_selection import train_test_split
# import vaex
# import mpl_scatter_density
# import matplotlib as mpl
# import matplotlib.pyplot as plt

# %matplotlib inline

# mpl.style.use('seaborn-colorblind')
# mpl.rc('font', size=15)
# mpl.rc('figure', figsize=(8, 6))

In [4]:
sim_dir = '/scratch/08317/tg876168/ananke_subsamples/m12i/lsr-0/'
sim_files = sorted(glob.glob(os.path.join(sim_dir, '*.hdf5')))
sim_files

['/scratch/08317/tg876168/ananke_subsamples/m12i/lsr-0/lsr-0-rslice-0.m12i-res7100-subsamples.hdf5',
 '/scratch/08317/tg876168/ananke_subsamples/m12i/lsr-0/lsr-0-rslice-1.m12i-res7100-subsamples.hdf5',
 '/scratch/08317/tg876168/ananke_subsamples/m12i/lsr-0/lsr-0-rslice-2.m12i-res7100-subsamples.hdf5']

In [5]:
for i_file in range(len(sim_files)):
    with h5py.File(sim_files[i_file], 'r') as f:
        print(list(f.keys()))
        print(f['ra'].len())

['b', 'dec', 'feh', 'l', 'labels', 'parallax', 'parallax_over_error', 'pmdec', 'pmra', 'px_true', 'py_true', 'pz_true', 'ra', 'radial_velocity', 'source_id', 'vx_true', 'vy_true', 'vz_true']
50683177
['b', 'dec', 'feh', 'l', 'labels', 'parallax', 'parallax_over_error', 'pmdec', 'pmra', 'px_true', 'py_true', 'pz_true', 'ra', 'radial_velocity', 'source_id', 'vx_true', 'vy_true', 'vz_true']
13581
['b', 'dec', 'feh', 'l', 'labels', 'parallax', 'parallax_over_error', 'pmdec', 'pmra', 'px_true', 'py_true', 'pz_true', 'ra', 'radial_velocity', 'source_id', 'vx_true', 'vy_true', 'vz_true']
2


In [6]:
keys = ('l', 'b', 'parallax', 'pmra', 'pmdec', 'radial_velocity','labels')

### Split data into train, validation, and test set

In [7]:
seed1 = 5642556
seed2 = 7196865

n_max_file = 10000    # 1M samples per file
# n_max_file = 10000000    # 10M samples per file

out_dir_base = '/scratch/08317/tg876168/dataset/m12i-lsr-0-test'

In [132]:
for p in keys:
    full_data = []
    for sim_file in sim_files:
        with h5py.File(sim_file, 'r') as input_f:
            full_data.append(input_f[p][:])
    full_data = np.concatenate(full_data)
    
    # split dataset into training and validation
    train_data, val_data = train_test_split(
        full_data, test_size=0.2, random_state=seed1, shuffle=True)
    val_data, test_data = train_test_split(
        val_data, test_size=0.5, random_state=seed2, shuffle=True)
    
    # write each dataset
    for data, flag in zip(
        (train_data, val_data, test_data), ('train', 'val', 'test')):
        n_file = data.shape[0] // n_max_file + 1
        
        out_dir = os.path.join(out_dir_base, flag)     
        os.makedirs(out_dir, exist_ok=True)
        
        for i in range(n_file):
            file = os.path.join(out_dir, 'n{:02d}.hdf5'.format(i))
            with h5py.File(file, 'a') as output_f:
                # write sliced dataset to file
                start = i * n_max_file
                end = (i + 1) * n_max_file
                output_f.create_dataset(p, data=data[start: end])

In [133]:
# print out the counts and fraction of in situ and accreted stars for each flag
# we want to make sure each flag is drawn from the same distribution

properties = {}

for flag in ('train', 'val', 'test'):
    files = sorted(glob.glob(os.path.join(out_dir_base, f'{flag}/*')))

    labels = []
    for file in files:
        with h5py.File(file, 'r') as f:
            labels.append(f['labels'][:])
    labels = np.concatenate(labels)
 
    n_insitu = np.sum(labels == 0)
    n_accreted = np.sum(labels == 1)
    n_total = len(labels)
    
    print(flag)
    print('Number of in situ samples   : {:d}'.format(n_insitu))
    print('Number of accreted samples  : {:d}'.format(n_accreted))
    print('Fraction of in situ samples : {:.4f}'.format(n_insitu / n_total))
    print('Fraction of accreted samples: {:.4f}'.format(n_accreted / n_total))
    print('Ratio accreted-in situ      : {:.4f}'.format(n_accreted / n_insitu))
    print('---------------')
    
    properties[flag] = {
        'n_insitu': int(n_insitu),
        'n_accreted': int(n_accreted),
        'n_total': int(n_total),
        'f_insitu': float(n_insitu / n_total),
        'f_accreted': float(n_accreted / n_total),
        'r_accreted_insitu': float(n_accreted / n_insitu)
    }

with open(os.path.join(out_dir_base, 'properties.json'), 'w') as f:
    json.dump(properties, f, indent=4)

train
Number of in situ samples   : 40245442
Number of accreted samples  : 311966
Fraction of in situ samples : 0.9923
Fraction of accreted samples: 0.0077
Ratio accreted-in situ      : 0.0078
---------------
val
Number of in situ samples   : 5030947
Number of accreted samples  : 38729
Fraction of in situ samples : 0.9924
Fraction of accreted samples: 0.0076
Ratio accreted-in situ      : 0.0077
---------------
test
Number of in situ samples   : 5030735
Number of accreted samples  : 38941
Fraction of in situ samples : 0.9923
Fraction of accreted samples: 0.0077
Ratio accreted-in situ      : 0.0077
---------------


### Preprocessing dict
During training, we will scale the training data such that for each feature, the mean and standard deviation is 0 and 1, respectively. We want to compute these values prior to training and store it somewhere such that it can be read easily (and quickly).

In [134]:
train_files = sorted(glob.glob(os.path.join(out_dir_base, 'train/*')))

preprocess_dict = {}

# loop over all keys and compute mean and stdv
for p in keys:
    if p == 'labels':
        continue
    data = []
    for file in train_files:
        with h5py.File(file, 'r') as f:
            data.append(f[p][:])
    data = np.concatenate(data)
    
    mean = np.nanmean(data)
    stdv = np.nanstd(data)
    
    print(f'Keys: {p}')
    print('Mean: {:.4e}'.format(mean))
    print('stdv: {:.4e}'.format(stdv))
    print('----------------')
    
    preprocess_dict[p] = {
        'mean': float(mean), 
        'stdv': float(stdv),
    }
    
with open(os.path.join(out_dir_base, 'preprocess.json'), 'w') as f:
    json.dump(preprocess_dict, f, indent=4)

Keys: l
Mean: 3.8857e-02
stdv: 9.2767e+01
----------------
Keys: b
Mean: 8.0702e-01
stdv: 3.1606e+01
----------------
Keys: parallax
Mean: 1.0820e+00
stdv: 1.0426e+00
----------------
Keys: pmra
Mean: -9.8260e-01
stdv: 2.2359e+01
----------------
Keys: pmdec
Mean: -2.7594e+00
stdv: 2.2913e+01
----------------
Keys: radial_velocity
Mean: -3.5690e+00
stdv: 6.8684e+01
----------------


### shuffle test

In [1]:
import torch
from torch.utils.data import DataLoader

In [7]:
# read in the labels
train_files = sorted(glob.glob(os.path.join(out_dir_base, 'train/*')))
with h5py.File(train_files[0], 'r') as f:
    labels = f['labels'][:]

Naive batching

In [12]:
batch_size = 1000
data_loader = DataLoader(labels, batch_size=batch_size)
for i, y in enumerate(data_loader):
    n0 = torch.sum(y == 0).numpy()
    n1 = torch.sum(y == 1).numpy()
    if n1 == 0:
        print(f'{i}')

2472
3468
4705
5369
7188
7885
8223


Weighted batching ensures the same number of data of each class in each batch

In [15]:
class_sample_count = np.array([len(np.where(labels==t)[0]) for t in np.unique(labels)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])

samples_weight = torch.from_numpy(samples_weight)
sampler = torch.utils.data.sampler.WeightedRandomSampler(
    samples_weight.type('torch.DoubleTensor'), len(samples_weight))

data_loader = DataLoader(labels, batch_size=batch_size, sampler=sampler)

In [16]:
for y in data_loader:
    n0 = torch.sum(y == 0)
    n1 = torch.sum(y == 1)
    print(n0, n1)
    break

tensor(495) tensor(505)


### Dataset test

In [2]:
import torch.utils.data
from torchvision import transforms, datasets

In [9]:
with open(os.path.join(out_dir_base, 'preprocess.json')) as f:
    preprocess_dict = json.load(f)

In [33]:

import os
import h5py
import glob

import numpy as np

import torch.utils.data

def sort_key(string):
    base = os.path.basename(string)
    base = os.path.splitext(base)[0]
    return int(base.split('_')[-1][1:])

class Dataset(torch.utils.data.Dataset):
    ''' Customized Dataset object for large dataset with multiple HDF5 files.
        Iterate over the input directory and read in only one HDF5 file at once to save memory.
        Can be passed directly into torch.utils.data.DataLoader.
    '''
    def __init__(self, input_dir, input_key, label_key=None, 
                 transform=None, shuffle=False, n_max_file=None):
        super().__init__()
        
        if isinstance(input_key, str):
            input_key = [input_key, ]

        # set attributes
        self.input_dir = input_dir
        self.input_key = input_key
        self.label_key = label_key
        self.shuffle = shuffle
        self.transform = transform

        self.input_dim = len(input_key)
    
        # get a list of file in directory
        self.input_files = sorted(
            glob.glob(os.path.join(input_dir, 'n*.hdf5')), key=sort_key)[:n_max_file]
        self.__check_keys(input_key)
        self.__check_keys(label_key)
        
        self.sizes = self.__get_sizes()
        self.sizes_c = np.cumsum(self.sizes)
        
        # set data cache
        self.cache = [None, None]
        self.current_file_index = None
    
    def __check_keys(self, key):
        ''' Iterate through self.input_files and check if keys exist '''
        if key is None:
            return
        
        for file in self.input_files:
            with h5py.File(file, 'r') as f:
                if isinstance(key, str):
                    if f.get(key) is None:
                        raise KeyError('Key {} does not exist in file {}'.format(key, file))
                else:
                    for k in key:
                        if f.get(k) is None:
                            raise KeyError('Key {} does not exist in file {}'.format(k, file))
      
    def __get_sizes(self):
        ''' Iterate through self.input_files and count the 
        number of samples in each file '''
        
        sizes = []
        for file in self.input_files:
            with h5py.File(file, 'r') as f:
                sizes.append(f[self.input_key[0]].len())
        return sizes
    
    def __len__(self):
        ''' Return number of samples '''
        return np.sum(self.sizes)
        
    def __cache(self, file_index):
        ''' Cache data '''
        # reset cache data
        self.cache = [None, None]
        
        with h5py.File(self.input_files[file_index], 'r') as f:
            # read in input data
            input_data = []
            for key in self.input_key:
                input_data.append(f[key][:])
            input_data = np.stack(input_data, 1)
                
            if self.shuffle:
                rand = np.random.permutation(input_data.shape[0])
                input_data = input_data[rand]
            self.cache[0] = input_data
            
            # read in label data
            if self.label_key is not None:
                label = f[self.label_key][:].reshape(-1, 1)
                if self.shuffle:
                    label = label[rand]
                self.cache[1] = label
    
    def __getitem__(self, index):
        ''' Get item of a given index. Index continuous between files 
        in the same order of self.input_files '''
        
        file_index = np.digitize(index, self.sizes_c)
        if file_index != self.current_file_index:
            self.__cache(file_index)
            self.current_file_index = file_index
        if file_index != 0:
            index -= self.sizes_c[file_index-1]
          
        if self.transform is not None:
            input_data = self.transform(self.cache[0][index])
        else:
            input_data = self.cache[0][index]
        
        if self.label_key is not None:
            labels = self.cache[1][index]
            return input_data, labels
        return input_data


In [35]:
input_key = ('l', 'b')
mean = [preprocess_dict[k]['mean'] for k in input_key]
std = [preprocess_dict[k]['stdv'] for k in input_key]

def transform(x):
    return (x - mean) / std
# transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean=mean, std=std)
#     ]
# )

train_dir = os.path.join(out_dir_base, 'train')
dataset = Dataset(
    train_dir, input_key, label_key='labels', transform=transform,
    shuffle=False, n_max_file=10)

In [36]:
len(dataset)

100000

In [42]:
dataset[5]

(array([-0.36792046, -0.35372509]), array([0]))

In [38]:
dataset.cache[0].shape, dataset.cache[1].shape

((10000, 2), (10000, 1))